# LA Wildfire Prediction: Model Evaluation



In [19]:


import pandas as pd
import numpy as np
import os
import joblib
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.metrics import accuracy_score, f1_score, classification_report, confusion_matrix
from sklearn.decomposition import PCA
from datetime import datetime

# For displaying plots in the notebook
%matplotlib inline


## Load Model and Feature Names
Load the trained model and feature names from saved files.


In [21]:
def load_model(model_path):
  
    print(f"Loading model from {model_path}...")
    model = joblib.load(model_path)
    return model


In [23]:
def load_feature_names(feature_path):
    
    print(f"Loading feature names from {feature_path}...")
    with open(feature_path, 'r') as f:
        feature_names = [line.strip() for line in f]
    return feature_names


## Load Test Data
Load the test data and prepare it for evaluation.


In [25]:
def load_test_data(data_path, feature_names):
  
    print(f"Loading test data from {data_path}...")
    df = pd.read_csv(data_path)
    df['date'] = pd.to_datetime(df['date'])
    
    # Print the head of loaded data
    print("\nLoaded Test Data Head:")
    print(df.head())
    
    # Create target variable based on Fire_Occurred
    if 'Fire_Occurred' in df.columns:
        y_test = df['Fire_Occurred']
        print("Using Fire_Occurred as target variable for evaluation")
        
        # Use the original features for PCA transformation later
        X_test = df[feature_names] if all(f in df.columns for f in feature_names) else df[
            [f for f in df.columns if f != 'Fire_Occurred' and f != 'date']]
        
        # Extract dates
        date_test = df['date']
    else:
        # Create a synthetic target if Fire_Occurred is not available
        if 'TMAX' in df.columns and 'days_since_rain' in df.columns:
            y_test = (df['TMAX'] / 100) * (df['days_since_rain'] / 30)
            X_test = df[feature_names] if all(f in df.columns for f in feature_names) else df[
                [f for f in df.columns if f != 'date']]
            date_test = df['date']
            print("Created synthetic target based on temperature and dryness for evaluation")
        else:
            # Create random target for demonstration
            y_test = np.random.rand(len(df))
            X_test = df[feature_names] if all(f in df.columns for f in feature_names) else df[
                [f for f in df.columns if f != 'date']]
            date_test = df['date']
            print("Created random target for demonstration")
    
    return X_test, y_test, date_test


## PCA Transformation
Apply Principal Component Analysis (PCA) to the test data to match the transformation applied during training.


In [27]:
def apply_pca_to_test_data(X_test, n_components=3):
   
    print(f"Applying PCA to test data with {n_components} components...")
    
    # Select only numeric columns
    numeric_cols = X_test.select_dtypes(include=['float64', 'int64']).columns
    X_numeric = X_test[numeric_cols]
    
    # Create PCA model
    pca = PCA(n_components=n_components)
    
    # Fit and transform the data
    X_pca_array = pca.fit_transform(X_numeric)
    
    # Create principal component column names
    component_names = [f'PC{i+1}' for i in range(X_pca_array.shape[1])]
    
    # Convert to DataFrame
    X_pca = pd.DataFrame(X_pca_array, columns=component_names)
    
    print(f"PCA reduced features from {X_numeric.shape[1]} to {X_pca.shape[1]} components.")
    
    return X_pca


## Model Evaluation
Evaluate the model performance using various metrics and visualizations.


In [29]:
def evaluate_model(model, X_test, y_test):
  
    print("Evaluating model performance...")
    
    # Make predictions
    y_pred = model.predict(X_test)
    
    # Calculate classification metrics
    acc = accuracy_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)
    
    print(f"Accuracy: {acc:.4f}")
    print(f"F1 Score: {f1:.4f}")
    
    print("\nClassification Report:")
    print(classification_report(y_test, y_pred, zero_division=1))
    
    # Calculate regression metrics
    mse = mean_squared_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)
    
    print(f"\nMean Squared Error: {mse:.4f}")
    print(f"R² Score: {r2:.4f}")
    
    # Plot confusion matrix
    conf_matrix = confusion_matrix(y_test, y_pred)
    plt.figure(figsize=(8, 6))
    sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues',
                xticklabels=['No Fire', 'Fire'],
                yticklabels=['No Fire', 'Fire'])
    plt.xlabel('Predicted')
    plt.ylabel('Actual')
    plt.title('Confusion Matrix')
    
    # Create directory if it doesn't exist
    os.makedirs('../reports/figures', exist_ok=True)
    plt.savefig('../reports/figures/confusion_matrix_evaluation.png')
    plt.close()
    
    metrics = {
        'accuracy': acc,
        'f1_score': f1,
        'mse': mse,
        'r2': r2
    }
    
    # Predict probabilities for the positive class (Fire = 1)
    y_pred_proba = model.predict_proba(X_test)[:, 1]
    
    # Save true labels, predicted labels, and probabilities
    predictions_df = pd.DataFrame({
        'True_Label': y_test,
        'Predicted_Label': y_pred,
        'Predicted_Probability': y_pred_proba
    })
    os.makedirs('reports', exist_ok=True)
    predictions_df.to_csv('reports/predicted_vs_true_labels.csv', index=False)
    print("Saved predicted vs true labels with probabilities to reports/predicted_vs_true_labels.csv")

    
    return metrics


## Feature Importance Analysis
Evaluate and visualize the importance of different features in the model.


In [31]:
def evaluate_feature_importance(model, feature_names):
    
    print("Evaluating feature importance...")
    
    try:
        # For pipeline with classifier step
        if hasattr(model, 'named_steps') and hasattr(model.named_steps.get('classifier', None), 'feature_importances_'):
            importances = model.named_steps['classifier'].feature_importances_
            feature_importance = pd.DataFrame({
                'Feature': feature_names,
                'Importance': importances
            }).sort_values('Importance', ascending=False)
        # For direct RandomForestClassifier
        elif hasattr(model, 'feature_importances_'):
            importances = model.feature_importances_
            feature_importance = pd.DataFrame({
                'Feature': feature_names,
                'Importance': importances
            }).sort_values('Importance', ascending=False)
        else:
            print("Model does not have feature_importances_ attribute.")
            return None
        
        # Print top features
        print("\nTop 10 Most Important Features:")
        print(feature_importance.head(10))
        
        # Visualize feature importance
        plt.figure(figsize=(12, 8))
        sns.barplot(x='Importance', y='Feature', data=feature_importance.head(15))
        plt.title('Feature Importance')
        plt.tight_layout()
        plt.savefig('../reports/figures/feature_importance_evaluation.png')
        plt.close()
        
        # Create a pie chart for the top 5 features
        plt.figure(figsize=(10, 10))
        top5_features = feature_importance.head(5)
        plt.pie(top5_features['Importance'], labels=top5_features['Feature'], 
                autopct='%1.1f%%', startangle=90, shadow=True)
        plt.axis('equal')  # Equal aspect ratio ensures that pie is drawn as a circle
        plt.title('Top 5 Features Contribution')
        plt.savefig('../reports/figures/top5_features_pie.png')
        plt.close()
        
        return feature_importance
    except Exception as e:
        print(f"Error evaluating feature importance: {e}")
        return None


## Seasonal Performance Analysis
Evaluate model performance across different seasons to identify seasonal patterns.


In [33]:
def evaluate_seasonal_performance(model, data_path, feature_names):
    
    print("Evaluating seasonal performance...")
    
    # Load data
    df = pd.read_csv(data_path)
    
    # Ensure date column is datetime
    if 'date' in df.columns:
        df['date'] = pd.to_datetime(df['date'])
        
        # Extract month and season
        df['month'] = df['date'].dt.month
        df['season'] = pd.cut(
            df['month'], 
            bins=[0, 3, 6, 9, 12], 
            labels=['Winter', 'Spring', 'Summer', 'Fall'],
            include_lowest=True
        )
        
        # Create target variable for evaluation
        if 'Fire_Occurred' in df.columns:
            y = df['Fire_Occurred']
            
            # Check if feature_names are PCA components
            if any(name.startswith('PC') for name in feature_names):
                # Use all columns except Fire_Occurred and date for PCA
                X_features = [col for col in df.columns if 
                             col != 'Fire_Occurred' and col != 'date' and col != 'month' and col != 'season']
                need_pca = True
            else:
                X_features = feature_names
                need_pca = False
        else:
            # Create a synthetic target
            if 'TMAX' in df.columns and 'days_since_rain' in df.columns:
                y = (df['TMAX'] / 100) * (df['days_since_rain'] / 30)
                X_features = feature_names
                need_pca = any(name.startswith('PC') for name in feature_names)
            else:
                # Create random target for demonstration
                y = np.random.rand(len(df))
                X_features = feature_names
                need_pca = any(name.startswith('PC') for name in feature_names)
        
        # Evaluate by season
        seasons = df['season'].unique()
        seasonal_metrics = []
        
        for season in seasons:
            season_df = df[df['season'] == season]
            X_season = season_df[X_features]
            y_season = y[season_df.index]
            
            # Apply PCA if needed
            if need_pca:
                n_components = sum(1 for name in feature_names if name.startswith('PC'))
                X_season = apply_pca_to_test_data(X_season, n_components)
            
            # Make predictions
            try:
                y_pred = model.predict(X_season)
                
                # Calculate metrics
                acc = accuracy_score(y_season, y_pred)
                f1 = f1_score(y_season, y_pred)
                
                # Add to results
                seasonal_metrics.append({
                    'Season': season,
                    'Accuracy': acc,
                    'F1 Score': f1,
                    'Sample Size': len(season_df)
                })
            except Exception as e:
                print(f"Error evaluating season {season}: {e}")
                continue
        
        # Convert to DataFrame
        seasonal_df = pd.DataFrame(seasonal_metrics)
        
        # Display results
        print("\nSeasonal Performance:")
        print(seasonal_df)
        
        # Visualize seasonal performance
        plt.figure(figsize=(12, 6))
        seasonal_df.set_index('Season')[['Accuracy', 'F1 Score']].plot(kind='bar')
        plt.title('Model Performance by Season')
        plt.ylabel('Score')
        plt.savefig('../reports/figures/seasonal_performance.png')
        plt.close()
        
        return seasonal_df
    else:
        print("Date column not found. Cannot evaluate seasonal performance.")
        return None


## Main Execution
Run the complete model evaluation pipeline.


In [35]:
def main():
    """Main function for evaluating the model."""
    # Define file paths
    model_path = "../models/la_fire_random_forest_model.pkl"
    feature_path = "../models/feature_names.txt"
    data_path = "../data/processed/engineered_la_fire_data.csv"
    
    # Create reports directory if it doesn't exist
    os.makedirs('../reports/figures', exist_ok=True)
    
    # Load model and feature names
    model = load_model(model_path)
    feature_names = load_feature_names(feature_path)
    
    # Load test data
    X_test, y_test, date_test = load_test_data(data_path, feature_names)
    
    # Check if we need to apply PCA
    if any(name.startswith('PC') for name in feature_names):
        # Count the number of PC components
        n_components = sum(1 for name in feature_names if name.startswith('PC'))
        # Apply PCA to test data
        X_test = apply_pca_to_test_data(X_test, n_components)
    
    # Evaluate model
    metrics = evaluate_model(model, X_test, y_test)
    
    # Evaluate feature importance
    feature_importance = evaluate_feature_importance(model, feature_names)
    
    # Evaluate seasonal performance
    seasonal_metrics = evaluate_seasonal_performance(model, data_path, feature_names)
    
    # Save evaluation results
    results = {
        'metrics': metrics,
        'top_features': feature_importance.head(10).to_dict() if feature_importance is not None else None,
        'seasonal_performance': seasonal_metrics.to_dict() if seasonal_metrics is not None else None
    }
    
    # Save as JSON
    import json
    with open('../reports/evaluation_results.json', 'w') as f:
        json.dump(results, f, indent=4)
    
    print("\nEvaluation results saved to reports/evaluation_results.json")
    print("Model evaluation completed successfully!")
    
    return metrics, feature_importance, seasonal_metrics


In [37]:
# Execute the model evaluation pipeline
metrics, feature_importance, seasonal_metrics = main()


Loading model from ../models/la_fire_random_forest_model.pkl...
Loading feature names from ../models/feature_names.txt...
Loading test data from ../data/processed/engineered_la_fire_data.csv...

Loaded Test Data Head:
        date  Fire_Occurred STATION NAME  AWND  DAPR  MDPR  PGTM  PRCP  TAVG  \
0 2014-12-27              0       0    0   0.0   0.0   0.0   0.0   0.0   0.0   
1 2014-12-28              0       0    0   0.0   0.0   0.0   0.0   0.0   0.0   
2 2014-12-29              0       0    0   0.0   0.0   0.0   0.0   0.0   0.0   
3 2014-12-30              0       0    0   0.0   0.0   0.0   0.0   0.0   0.0   
4 2014-12-31              0       0    0   0.0   0.0   0.0   0.0   0.0   0.0   

   ...  season_Summer  season_Fall  TMAX_3D  TMIN_3D  PRCP_3D  PRCP_14D  \
0  ...          False         True      0.0      0.0      0.0       0.0   
1  ...          False         True      0.0      0.0      0.0       0.0   
2  ...          False         True      0.0      0.0      0.0       0.0   
3